In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('/kaggle/input/movie-lens-2/u.data', sep='\t', names=column_names) #we have a tab seperated data hence we use '\t' to seperate it

In [ ]:
df.head()

We have the ID of the user who gave the rating to the movie, the movie ID and the timestamp of the movie  where it was rated. Lets  get the Titles for the respective title IDs.

In [ ]:
movie_titles = pd.read_csv('/kaggle/input/movie-lens-2/Movie_Id_Titles') #getting the movie titles

In [ ]:
movie_titles.head()

Lets merge this with our main data frame so that we can easily map the titles and the Item IDs

In [ ]:
df = pd.merge(df,movie_titles,on='item_id')
df.head()

We have merged both the data frames.Lets Visualize our data.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')

We will now take a look at the movie titles with most ratings and the titles that were rated maximum number of times by the viewers.

In [ ]:
df.groupby('title')['rating'].mean().sort_values(ascending=False).head()

In [ ]:
df.groupby('title')['rating'].count().sort_values(ascending=False).head()

We did this to compare something. It is possible that only a few people saw one of the top rated movies and gave it a 5-star rating but the truly highly rated movies are those that were watched by many and were givem 5-star ratings.

In [ ]:
ratings = pd.DataFrame(df.groupby('title')['rating'].mean())
ratings.head()

Lets also include the number of ratings for each title so that we can get better understanding of user data

In [ ]:
ratings['num of ratings'] = pd.DataFrame(df.groupby('title')['rating'].count())
ratings.head()

In [ ]:
plt.figure(figsize=(10,4))
ratings['num of ratings'].hist(bins=70)

We can see that most of our movies have a very few ratings, this ccan be because of the reason that people tend to watch movies that are more famous and more talked about.

In [ ]:
plt.figure(figsize=(10,4))
ratings['rating'].hist(bins=70)

We can see tha distribution in the ratings in our data set in the above plot

In [ ]:
sns.jointplot(x='rating',y='num of ratings',data=ratings,alpha=0.5)

This shows us the relation between our ratings and the number of ratings according to the data

In [ ]:
moviemat = df.pivot_table(index='user_id',columns='title',values='rating')
moviemat.head()

In [ ]:
ratings.sort_values('num of ratings',ascending=False).head(10)

Lets grab user ratings for Star wars and Liar Liar

In [ ]:
starwars_user_ratings = moviemat['Star Wars (1977)']
liarliar_user_ratings = moviemat['Liar Liar (1997)']

Lets get the correlation between the two Pandas series

In [ ]:
similar_to_starwars = moviemat.corrwith(starwars_user_ratings)
similar_to_liarliar = moviemat.corrwith(liarliar_user_ratings)

Our result has some Nan Values, so lets get rid of them and convert the series into a data frame

In [ ]:
corr_starwars = pd.DataFrame(similar_to_starwars,columns=['Correlation'])
corr_starwars.dropna(inplace=True)
corr_starwars.head()

Lets take a look at some of the results that are not quite satisfactory

In [ ]:
corr_starwars.sort_values('Correlation',ascending=False).head(10)

Here we see some movies that have perfect correlation with Star wars. You might have never heard about these before but it still shows the result. It might be due to the fact that there could be  a person who saw Star wars and gave it a 5-star rating and also saw on of these above movies and gave it a 5-star rating

In [ ]:
corr_starwars = corr_starwars.join(ratings['num of ratings'])
corr_starwars.head()

We took a look at the number of ratings, so lets filter out our previous results and consider only those movies that have more than 100 reviews.

In [ ]:
corr_starwars[corr_starwars['num of ratings']>100].sort_values('Correlation',ascending=False).head()

As you can see this helped us a lot in narrowing down our result as you can see movies that are similar to the Star Wars movie and there some pretty famous name in the above list. Lets do the same thing for Liar Liar which is a movie from the comedy genre

In [ ]:
corr_liarliar = pd.DataFrame(similar_to_liarliar,columns=['Correlation'])
corr_liarliar.dropna(inplace=True)
corr_liarliar = corr_liarliar.join(ratings['num of ratings'])
corr_liarliar[corr_liarliar['num of ratings']>100].sort_values('Correlation',ascending=False).head()

And with this final result we have completed our Basic Recommender system that one needs to know before going in deep in the subject of Recommender Systems where we use collaborative filtering and SVD.